In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Loading Reviews to Dataframe...
print('Loading reviews...',end='')
df = pd.read_csv('train.csv')
print('[ok]')
print('Loaded {:,} reviews' .format(len(df)))

In [ ]:
df.columns #Deciding on Columns we might need, dropping the rest for now

In [ ]:
""" 
We Found relatively Small number of NAN onject in the dataset  in both TEXT and Summary features
wE will replace with '' and drop the UN-NEEDED columns for now
"""
df.Summary.fillna('', inplace=True)
df.Text.fillna('', inplace=True)
df.drop(['Id','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Time','ProductId'],axis=1,inplace=True)

In [ ]:
df.columns

# 1- Data Exploration and Visualization: (10%)
Provide an initial step to inspect, visualize and analyse the different attributes in your data set.
Document your findings and make conclusions for your next steps

In [ ]:
#Concatinating TEXT and Summary features as per the DR. example. Which is OK as I didnt find obvious correlation
df['text']=df['Summary']+' '+df['Text'] 
df.drop(['Summary','Text'],axis=1,inplace=True)
df['length']=df['text'].str.len() 

In [ ]:
df.columns

In [ ]:
df.hist()
# We need to comment on the graph below, Basically the data is biased towards score 5 . 

In [ ]:
df.groupby('Score').describe()

In [ ]:
# We can conclude from the table above, that the distributions is not equal, text with ratings 5 are far more than any other score

In [ ]:
df.hist(column='length',by='Score',bins=200,figsize=(8,15))

In [ ]:
# We need to write the conclustion for the graph
#ALso feel free to add whatever graph you like so we can visualize the data as much as we can
# DataFrame is called df , you can see its columns by running df.columns for more info run  df.info(), df.describe() ...etc

# 2 - Text Processing and Normalization: (20%)
Thoroughly experiment with different text processing and normalization alternatives. Explain the
trade-off and benefits of using each and justify their effectiveness for the current data set.

"""
NOTE FOR THE TEAM
This question (Question 2) is all about text normalization (tokenization) , this includes
cleaning the text and make it uniform, this is mainly done via removing punctuations, stopword
After our call, I noticed some html tags in the text, we need to remove them as well.
The catch is they MUST be removed before the pre-processing function , the reason is once we use the function, we loose punctuations, 
so <br> looks like br , so we cant really figure out that it was in fact and HTML tag.
SO I will work on creating a fn to clean the html tags first, then pass it to the tokenization function 
"""

"""
Update, I found a libraries that can do that, you need to install 
pip install lxml
pip install beautifulsoup4
Added them to the tokenizer function
"""

In [ ]:
# We will use 2 methods to tokeinze the corpse...stemming and Lemmatization , both from NLTK librarry

#Once we finzalie the analysis, we will use CV and TFIDF to vectorize then build the pipeline with either one of them (cant have both in pipline AFAIK)
# (possibly we will use TFIDF)

#UPDATE, we will only remove the html tags and puncations ( we wont test that i believe as this will make the model better)


In [ ]:
#Method 1 - Building our own tokenizer (to remove HTML and punc)

In [ ]:
import string,lxml,bs4


In [ ]:
string.punctuation # we will remove this


In [ ]:
#This function will remove html tags, punctiaton [Must run in order ]
def tokenizer(text):
    no_html = bs4.BeautifulSoup(text,'lxml').get_text()
    no_punctuation = [char for char in no_html if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return no_punctuation

In [ ]:
df_original = df.copy() #Backup to work on raw input data if we will need it later...

In [ ]:
# IMPORTNAT... THIS IS CREATING A SMALLER SUBSET OF DATA, REMOVE THIS LINE TO WORK ON ALL DATA
df = df.head(1000)
print ( 'Original DB size is',len(df_original),' Current db size',len(df))

In [ ]:
df['text'] = df['text'].apply(tokenizer)

### Stemming 

Port Stemmer

In [ ]:
df_PS = df.copy()

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
nltkstemmer = PorterStemmer()

In [ ]:
def stemmer(text):
    review = [nltkstemmer.stem(word) for word in text.split()]
    review = ' '.join(review) 
    return review
    
    

In [ ]:
df_PS['text']= df_PS['text'].apply(stemmer)

### Lemmatizing

In [ ]:
# decided to go with option v, not n for no obvoius reason except my surface laptop is burning hot :)!

In [ ]:
df_LM = df.copy()

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
nltklem = WordNetLemmatizer()

In [ ]:
def lemmatizer(text):
    review = [nltklem.lemmatize(word,'v') for word in text.lower().split()]
    review = ' '.join(review)
    #print(review)
    return review

In [ ]:
df_LM['text'] = df_LM['text'].apply(lemmatizer)

#### Comparison of the 3 datasets 

In [ ]:
i=15 #set i to the record you wanna see

In [ ]:
df_original['text'].iloc[i]

In [ ]:
df['text'].iloc[i] 

In [ ]:
df_PS['text'].loc[i]

In [ ]:
df_LM['text'].loc[i]

### Conclusion 

In [ ]:
# Write our conclusion here...!











# Vector space Model and feature representation: (20%)
Experiment with different representation techniques. Document your findings and make
conclusions. Show how choosing n-gram features can influence your results

### CountVictorizer 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Method1 df | CountVectorizer (defualt 'word' analyzer)

# Fitting 
bow_m1 = CountVectorizer().fit(df['text'])
#Transform 
df_small_bow_m1 = bow_m1.transform(df['text'])

In [ ]:
# Method2 df | CountVectorizer(our custom analyzer aka Tokenizer)

# Fitting 
bow_m2 = CountVectorizer(analyzer=tokenizer).fit(df['text'])
#Transform 
df_small_bow_m2 = bow_m2.transform(df['text'])

In [ ]:
# Method3 df | CountVectorizer(char analyzer)

# Fitting 
bow_m3 = CountVectorizer(analyzer='char').fit(df['text'])
#Transform 
df_small_bow_m3 = bow_m3.transform(df['text'])

In [ ]:
# Method4 df | CountVectorizer(stop words )

# Fitting 
bow_m4 = CountVectorizer(stop_words={'english'},analyzer= 'word',ngram_range=(1,2)).fit(df['text'])
#Transform 
df_small_bow_m4 = bow_m4.transform(df['text'])

In [ ]:
#findings and Results

In [ ]:
print('Method 1 CV')
print('Bag of words CV1',len(bow_m1.vocabulary_))


In [ ]:
print('Method 2 CV')
print('Bag of words CV2',len(bow_m2.vocabulary_))

In [ ]:
print('Method 3 CV')
print('Bag of words CV3',len(bow_m3.vocabulary_))

In [ ]:
print('Method 4 CV')
print('Bag of words CV4',len(bow_m4.vocabulary_))

### TFIDF



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
BOW_TFIDF1 = TfidfVectorizer(stop_words={'english'},ngram_range=(1,2))
BOW_TFIDF1.fit(df['text'])
#BOW_TFIDF1.fit_transform(df_small['text'])

In [ ]:
BOW_TFIDF2 = TfidfVectorizer(analyzer=tokenizer,ngram_range=(1,2))
BOW_TFIDF2.fit(df['text'])
#BOW_TFIDF1.fit_transform(df_small['text'])

In [ ]:
BOW_TFIDF3 = TfidfVectorizer(ngram_range=(1,2),min_df=2)
BOW_TFIDF3.fit(df['text'])
#BOW_TFIDF1.fit_transform(df_small['text'])

In [ ]:
print('Method 1 TFIDF')
print('Bag of words TFIDF1',len(BOW_TFIDF1.vocabulary_))

In [ ]:
print('Method 2 TFIDF')
print('Bag of words TFIDF2',len(BOW_TFIDF2.vocabulary_))

In [ ]:
print('Method 3 TFIDF')
print('Bag of words TFIDF3',len(BOW_TFIDF3.vocabulary_))

# Model training, selection and hyperparameter tuning and evaluation:(20%)
You should at least experiment with 3 models and show how you can optimize model
parameters using cross validation. For each model discuss your choices of text processing,
representation and features from steps 1-3.